En este apartado, llevaremos a cabo el entrenamiento de nuestro modelo de Machine Learning para armar un sistema de recomendación.

El modelo tendrá una relación ítem-ítem; es decir, se tomará un ítem y, en base a qué tan similar es al resto, se recomendarán similares. Aquí el input es un juego y el output es una lista de juegos recomendados, para ello se aplicará la similitud del coseno.

### 1. Importación de las bibliotecas necesarias

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

### 2. Carga de Dataset

In [2]:
df_steam_games = pd.read_csv("../Archivos_Limpios/output_steam_games_cleaned.csv")

### 3. Tratamiento adicional a los datos


In [3]:
# Definimos una función llamada preprocess que toma un texto como entrada
def preprocess(text):
    # Reemplaza cualquier caracter que no sea una letra (mayúscula o minúscula) ni un número por un espacio en blanco
    text_input = re.sub('[^a-zA-Z1-9]+', ' ', str(text))
    # Elimina todos los dígitos del texto
    output = re.sub(r'\d+', '', text_input)
    # Convertir el texto resultante a minúsculas y eliminar espacios en blanco al principio y al final
    return output.lower().strip()

In [4]:
# Aplicamos la función preprocess a la columna specs del DataFrame
df_steam_games["specs"] = df_steam_games.specs.map(preprocess)

### 4. El Modelo

Creación de un Vectorizador de Texto (CountVectorizer):

- Propósito: Convertir texto (en este caso, la columna "specs") en vectores numéricos.

- Proceso: El vectorizador asigna un número a cada palabra única presente en el texto y cuenta su frecuencia. Cada documento (en este caso, cada juego) se representa como un vector donde cada posición corresponde a la frecuencia de una palabra.

- Importancia: Esta representación vectorial es necesaria para calcular la similitud del coseno, ya que este método trabaja con vectores numéricos.

In [23]:
# Creamos un vectorizador de texto
cv = CountVectorizer()
vectores = cv.fit_transform(df_steam_games["specs"]).toarray()

Cálculo de la Similitud del Coseno entre Vectores:

- Propósito: Evaluar la similitud entre los vectores numéricos de diferentes juegos.

- Proceso: Se utiliza la métrica de similitud del coseno, que mide el coseno del ángulo entre dos vectores. Cuanto más cercano a 1, más similares son los vectores.

- Importancia: La similitud del coseno es clave para determinar qué tan parecidos son los juegos entre sí. Esto se utiliza para generar recomendaciones, ya que los juegos con vectores similares son considerados como recomendaciones potenciales.

In [ ]:
# Calculamos la similitud del coseno entre vectores
similitud = cosine_similarity(vectores)

A continuación, se genera una función (recomendacion) que toma como entrada el ID de un juego y devuelve una lista de los cinco juegos más recomendados basándose en la similitud del coseno entre vectores.La función devuelve la lista de títulos recomendados.

In [25]:
# Generamos una función para obtener recomendaciones por título
def recomendacion(juego):

    # Buscamos el índice del juego en el DataFrame.
    # Este índice es utilizado para acceder a la fila correspondiente en la matriz de similitud.
    indice_juego = df_steam_games[df_steam_games["item_id"] == juego].index[0]
    
    # Cálculo de Similitudes:
    # Se obtienen las distancias de similitud entre el juego de entrada y todos los demás juegos en el conjunto de datos. 
    distances = similitud[indice_juego]
    
    # Las distancias se ordenan de manera descendente, y se seleccionan los cinco juegos más similares (excluyendo el juego de entrada)
    lista_juegos = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    
    # Salida. Devuelve la lista de títulos recomendados.
    recommended_titles = [df_steam_games.iloc[i[0]]["name"] for i in lista_juegos]
    
    return recommended_titles

In [ ]:
# Aplicamos la función a la columna "item_id" y creamos una nueva columna "Recomendaciones"
df_steam_games["Recomendaciones"] = df_steam_games["item_id"].apply(recomendacion)

In [ ]:
# Eliminamos columnas innecesarias para disminuir el tamaño del archivo de salida
df_steam_games.drop(columns=["genres", "name", "specs", "price", "developer", "year"], inplace=True)
df_steam_games

In [ ]:
# Guardamos el DataFrame resultante en un nuevo archivo CSV
df_steam_games.to_csv('Modelo_ML.csv', index=False)